In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os

In [2]:
SEARCH_QUERIES = ["altocumulus clouds"]
max_images = 10

In [3]:
url = f"https://duckduckgo.com/?q={SEARCH_QUERIES[0].replace(' ', '+')}&t=h_&iax=images&ia=images"
url

'https://duckduckgo.com/?q=altocumulus+clouds&t=h_&iax=images&ia=images'

In [4]:
response = requests.get(url)

In [5]:
html = response.content
html

b'<!DOCTYPE html><html lang="en-US" class="no-js has-zcm  no-theme is-link-style-exp is-link-order-exp is-link-breadcrumb-exp is-related-search-exp is-vertical-tabs-exp  "><head><meta name="description" content="DuckDuckGo. Privacy, Simplified."><meta http-equiv="content-type" content="text/html; charset=utf-8"><title>altocumulus clouds at DuckDuckGo</title><meta name="apple-itunes-app" content="app-id=663592361, app-argument=https://duckduckgo.com/?q=altocumulus+clouds&t=h_&iax=images&ia=images&smartbanner=1"><link rel="stylesheet" href="/dist/s.27f5ee0a705a2c7a4228.css" type="text/css"><link rel="stylesheet" href="/dist/r.b5bdfacdd5d026b46388.css" type="text/css"><link rel="stylesheet" href="/dist/wpl.main.0285490bc835b4bdcbd7.css" type="text/css" data-handle-css-custom-properties="true"><link rel="stylesheet" href="/dist/wpl.vendors.f96ec62b8a9c623d21f2.css" type="text/css" data-handle-css-custom-properties="true"><meta name="robots" content="noindex,nofollow"><meta name="referrer" 

In [6]:
soup = bs(html, "html.parser")

In [7]:
soup

<!DOCTYPE html>
<html class="no-js has-zcm no-theme is-link-style-exp is-link-order-exp is-link-breadcrumb-exp is-related-search-exp is-vertical-tabs-exp" lang="en-US"><head><meta content="DuckDuckGo. Privacy, Simplified." name="description"/><meta content="text/html; charset=utf-8" http-equiv="content-type"/><title>altocumulus clouds at DuckDuckGo</title><meta content="app-id=663592361, app-argument=https://duckduckgo.com/?q=altocumulus+clouds&amp;t=h_&amp;iax=images&amp;ia=images&amp;smartbanner=1" name="apple-itunes-app"/><link href="/dist/s.27f5ee0a705a2c7a4228.css" rel="stylesheet" type="text/css"/><link href="/dist/r.b5bdfacdd5d026b46388.css" rel="stylesheet" type="text/css"/><link data-handle-css-custom-properties="true" href="/dist/wpl.main.0285490bc835b4bdcbd7.css" rel="stylesheet" type="text/css"/><link data-handle-css-custom-properties="true" href="/dist/wpl.vendors.f96ec62b8a9c623d21f2.css" rel="stylesheet" type="text/css"/><meta content="noindex,nofollow" name="robots"/><m

In [8]:
soup.title.get_text()

'altocumulus clouds at DuckDuckGo'

In [9]:
soup.find_all("img")

[]

In [10]:
[SEARCH_QUERIES]

[['altocumulus clouds']]

In [11]:
def search_images(query: str, max_images: int = 10, search_engine: str = "bing") -> list[str]:
    """
    Search for images using the specified search engine and return a list of image URLs.

    Args:
        query (str): The search query for images.
        max_images (int): The maximum number of image URLs to return.
        search_engine (str): The search engine to use ('bing' is currently supported).

    Returns:
        list[str]: A list of image URLs.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    if isinstance(query, str):
        query = [query]

    image_urls = []
    for i, q in enumerate(query):

        q = q.replace(" ", "+")

        urls = {
            "bing": f"https://www.bing.com/images/search?q={q}",
        }

        classes = {
            "bing": "iusc",
        }

        response = requests.get(urls[search_engine], headers=headers)
        soup = bs(response.text, "html.parser")

        images_elements = soup.find_all("a", class_=classes[search_engine])

        for img in images_elements:
            m_json = img.get("m")
            if m_json:
                try:
                    import json
                    m_data = json.loads(m_json)
                    image_urls.append(m_data["murl"])
                    if len(image_urls) >= max_images * i:  
                        break
                except json.JSONDecodeError:
                    continue
    return image_urls

In [12]:
urls = search_images(["cumulus clouds", "cirrus clouds"], max_images=10)

for i, url in enumerate(urls):
    print(f"Image {i+1}")

Image 1
Image 2
Image 3
Image 4
Image 5
Image 6
Image 7
Image 8
Image 9
Image 10


In [13]:
from cloudproject import search_images

ImportError: cannot import name 'search_images' from 'cloudproject' (/Users/corentinpradier/Documents/Codes/cloud-project/src/cloudproject/__init__.py)

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time

def search_image_urls(query: str, max_images: int = 10) -> list[str]:
    """
    Recherche des images sur Bing et retourne une liste d'URLs.
    Gère le défilement infini pour obtenir le nombre d'images souhaité.

    Args:
        query (str): Le terme de recherche (ex: "cumulus clouds").
        max_images (int): Le nombre maximum d'URLs d'images à retourner.

    Returns:
        list[str]: Une liste d'URLs d'images.
    """
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")

    driver = None
    image_urls = set()
    try:
        driver = webdriver.Chrome(options=options)
        search_url = f"https://www.bing.com/images/search?q={query.replace(' ', '+')}"
        print(f"Navigation vers : {search_url}")
        driver.get(search_url)

        last_height = driver.execute_script("return document.body.scrollHeight")

        while len(image_urls) < max_images:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Attendre que la page charge les nouvelles images

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("Fin de la page atteinte.")
                break
            last_height = new_height

            thumbnail_elements = driver.find_elements(By.CLASS_NAME, "iusc")
            for thumb in thumbnail_elements:
                m_json = thumb.get_attribute("m")
                if m_json:
                    try:
                        m_data = json.loads(m_json)
                        image_urls.add(m_data["murl"])
                    except (json.JSONDecodeError, KeyError):
                        continue
            print(f"URLs uniques trouvées : {len(image_urls)}/{max_images}")

    except Exception as e:
        print(f"Une erreur est survenue lors du scraping : {e}")
    finally:
        if driver:
            driver.quit()

    print(f"Scraping terminé. {len(image_urls)} URLs uniques trouvées.")
    return list(image_urls)[:max_images]


In [ ]:
def search_image_urls(query: str, max_images: int = 10, verbose: bool = False) -> list[str]:
    """
    Recherche des images sur Bing et retourne une liste d'URLs.
    S'arrête si le nombre d'images récupérées est le même 3 fois d'affilée.

    Args:
        query (str): Le terme de recherche (ex: "cumulus clouds").
        max_images (int): Le nombre maximum d'URLs d'images à retourner.

    Returns:
        list[str]: Une liste d'URLs d'images.
    """
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")

    driver = None
    image_urls = set()
    try:
        driver = webdriver.Chrome(options=options)
        search_url = f"https://www.bing.com/images/search?q={query.replace(' ', '+')}"
        if verbose:
            print(f"Navigation vers : {search_url}")
        driver.get(search_url)

        last_height = driver.execute_script("return document.body.scrollHeight")

        scroll_attempts_without_change = 0
        MAX_ATTEMPTS = 3

        same_count_attempts = 0  # compteur de stagnation
        previous_count = 0       # nombre d'images lors de la précédente boucle

        while len(image_urls) < max_images:
            # Extraire les URLs
            thumbnail_elements = driver.find_elements(By.CLASS_NAME, "iusc")
            for thumb in thumbnail_elements:
                m_json = thumb.get_attribute("m")
                if m_json:
                    try:
                        m_data = json.loads(m_json)
                        image_urls.add(m_data["murl"])
                    except (json.JSONDecodeError, KeyError):
                        continue

            current_count = len(image_urls)
            if verbose:
                print(f"URLs uniques trouvées : {current_count}/{max_images}")

            # Vérifier si le nombre d'images a augmenté
            if current_count == previous_count:
                same_count_attempts += 1
                if verbose:
                    print(f"Nombre d'images inchangé ({current_count}) - tentative {same_count_attempts}/3")
                if same_count_attempts >= 3:
                    if verbose:
                        print("Nombre d'images inchangé trois fois d'affilée. Arrêt.")
                    break
            else:
                same_count_attempts = 0  # reset si ça évolue
                previous_count = current_count

            if current_count >= max_images:
                if verbose:
                    print(f"Objectif de {max_images} URLs atteint.")
                break

            # Scroll vers le bas
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts_without_change += 1
                if verbose:
                    print(f"La hauteur de la page n'a pas changé (tentative {scroll_attempts_without_change}/{MAX_ATTEMPTS}).")
                try:
                    see_more_button = driver.find_element(By.CLASS_NAME, "btn_seemore")
                    if verbose:
                        print("Bouton 'Voir plus' trouvé, clic...")
                    driver.execute_script("arguments[0].click();", see_more_button)
                    time.sleep(2)
                    scroll_attempts_without_change = 0
                except Exception:
                    if scroll_attempts_without_change >= MAX_ATTEMPTS:
                        if verbose:
                            print("Fin de page ou plus de contenu détecté.")
                        break
            else:
                scroll_attempts_without_change = 0
            last_height = new_height

    except Exception as e:
        if verbose:
            print(f"Une erreur est survenue lors du scraping : {e}")
    finally:
        if driver:
            driver.quit()

    print(f"Scraping terminé. {len(image_urls)} URLs uniques trouvées.")
    return list(image_urls)[:max_images]

In [ ]:
from typing import Union
from PIL import Image
import io
import requests

def url_to_image(url: Union[str, list[str]]):
    if isinstance(url, str):
        url = [url]

    images = []
    for u in url:
        response = requests.get(u, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        image = Image.open(io.BytesIO(response.content))
        images.append(image)
    
    return images

In [ ]:
urls = search_image_urls("altocumulus clouds", max_images=10, verbose=True)

images, filenames = url_to_image(urls[:5], save=True)  # Affiche les 5 premières images

Navigation vers : https://www.bing.com/images/search?q=altocumulus+clouds
URLs uniques trouvées : 50/10
Objectif de 10 URLs atteint.
Scraping terminé. 50 URLs uniques trouvées.
Image sauvegardée sous : altocumulus.jpg
Image sauvegardée sous : large.5768dda1ccb6d_Ac1.jpg
Image sauvegardée sous : IMG_1902-1-600x400.jpg
Image sauvegardée sous : altostratus-cloud-classification-middle-level-cloud4.jpg
Image sauvegardée sous : altocumulus-sunset-krista-wimmer.jpg


In [ ]:
filenames

['altocumulus.jpg',
 'large.5768dda1ccb6d_Ac1.jpg',
 'IMG_1902-1-600x400.jpg',
 'altostratus-cloud-classification-middle-level-cloud4.jpg',
 'altocumulus-sunset-krista-wimmer.jpg']

In [ ]:
from pathlib import Path

def url_to_image(url: Union[str, list[str]], save: bool = False, save_path: str = "../../data/scraped/") -> list[Image.Image]:
    if isinstance(url, str):
        url = [url]

    images = []
    filenames = []
    for u in url:
        response = requests.get(u, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        image = Image.open(io.BytesIO(response.content))
        images.append(image)

        if save:
            filename = clean_image_filename(u)
            filenames.append(filename)
            image.save(f"{save_path}{filename}")
            print(f"Image sauvegardée sous : {filename}")

    if save:
        return images, filenames
    else:
        return images


In [ ]:
import os
import re
from urllib.parse import urlparse

def clean_image_filename(url: str) -> str:
    """
    Extrait le nom de fichier propre à partir d'une URL d'image.
    Supprime les paramètres après l'extension (.jpg, .png, etc.).

    Args:
        url (str): L'URL de l'image.

    Returns:
        str: Le nom de fichier propre, ou None si aucun nom valide trouvé.
    """
    # Extraire le chemin brut de l'URL
    parsed = urlparse(url)
    path = parsed.path  # exemple : "/images/photo.jpg"

    # Extraire le dernier segment (le nom du fichier brut)
    filename = os.path.basename(path)

    # Nettoyer : supprimer tout après l'extension d'image
    match = re.match(r"^(.*?\.(jpg|jpeg|png|gif|bmp|webp))", filename, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None
